# Qulacs & openfermion & psi4 で VQE

必要なもの

- qulacs
- openfermion
- openfermion-psi4
- psi4
- scipy
- numpy

In [1]:
import sys, os
import qulacs

from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.transforms import get_sparse_operator
from openfermion.hamiltonians import MolecularData
from openfermionpsi4 import run_psi4

from scipy.optimize import minimize
import numpy as np
import matplotlib.pyplot as plt

## ハミルトニアンを作る

In [2]:
basis = "sto-3g"
multiplicity = 1
charge = 0
trotter_step = 1
distance  = 0.977
geometry = [["H", [0,0,0]],["H", [0,0,distance]]]
description  = "tmp"
molecule = MolecularData(geometry, basis, multiplicity, charge, description)
molecule = run_psi4(molecule,run_scf=1,run_fci=1)

n_qubit = molecule.n_qubits
n_electron = molecule.n_electrons

### ハミルトニアンを変換・表示する

In [3]:
fermionic_hamiltonian = get_fermion_operator(molecule.get_molecular_hamiltonian())
jw_hamiltonian = jordan_wigner(fermionic_hamiltonian)

In [4]:
print(jw_hamiltonian)

(-0.31349601806504035+0j) [] +
(-0.04883472643300806+0j) [X0 X1 Y2 Y3] +
(0.04883472643300806+0j) [X0 Y1 Y2 X3] +
(0.04883472643300806+0j) [Y0 X1 X2 Y3] +
(-0.04883472643300806+0j) [Y0 Y1 X2 X3] +
(0.13978238244930488+0j) [Z0] +
(0.1576263053229024+0j) [Z0 Z1] +
(0.1074538256834896+0j) [Z0 Z2] +
(0.15628855211649767+0j) [Z0 Z3] +
(0.13978238244930494+0j) [Z1] +
(0.15628855211649767+0j) [Z1 Z2] +
(0.1074538256834896+0j) [Z1 Z3] +
(-0.13686894969530392+0j) [Z2] +
(0.16419290083579907+0j) [Z2 Z3] +
(-0.1368689496953039+0j) [Z3]


### ハミルトニアンを qulacs ハミルトニアンに変換する

In [24]:
from qulacs import Observable
from qulacs.observable import create_observable_from_openfermion_text

In [26]:
qulacs_hamiltonian = create_observable_from_openfermion_text(str(jw_hamiltonian))

In [6]:
file_name = 'VQE_H2_jw_hamiltonian.txt'
with open(file_name, mode='w') as f:
    f.write(str(jw_hamiltonian))

In [7]:
qulacs_hamiltonian = Observable(file_name)

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. qulacs.Observable(arg0: int)

Invoked with: 'VQE_H2_jw_hamiltonian.txt'

In [ ]:
# def convert_openfermion_op(n_qubit, openfermion_op):
#     """convert_openfermion_op

#     Args:
#         n_qubit (:class:`int`)
#         openfermion_op (:class:`openfermion.ops.QubitOperator`)
#     Returns:
#         :class:`qulacs.Observable`
#     """
#     ret = Observable(n_qubit)
    
#     for pauli_product in openfermion_op.terms:
#         coef = float(np.real(openfermion_op.terms[pauli_product]))
#         pauli_string = ''
#         for pauli_operator in pauli_product:
#             pauli_string += pauli_operator[1] + ' ' + str(pauli_operator[0])
#             pauli_string += ' '
#         ret.add_operator(coef, pauli_string[:-1])
    
#     return ret

In [ ]:
# qulacs_hamiltonian = convert_openfermion_op(n_qubit, jw_hamiltonian)

## ansatz を構成する

In [27]:
from qulacs import QuantumState, QuantumCircuit
from qulacs.gate import CZ, RY, RZ, merge

depth = n_qubit

In [28]:
def he_ansatz_circuit(n_qubit, depth, theta_list):
    """he_ansatz_circuit
    Retunrs hardware efficient ansatz circuit.

    Args:
        n_qubit (:class:`int`):
            the number of qubit used (equivalent to the number of fermionic modes)
        depth (:class:`int`):
            depth of the circuit.
        theta_list (:class:`numpy.ndarray`):
            rotation angles.
    Returns:
        :class:`qulacs.QuantumCircuit`
    """
    circuit = QuantumCircuit(n_qubit)
    
    for d in range(depth):
        for i in range(n_qubit):
            circuit.add_gate(merge(RY(i, theta_list[2*i+2*n_qubit*d]), RZ(i, theta_list[2*i+1+2*n_qubit*d])))
        for i in range(n_qubit//2):
            circuit.add_gate(CZ(2*i, 2*i+1))
        for i in range(n_qubit//2-1):
            circuit.add_gate(CZ(2*i+1, 2*i+2))
    for i in range(n_qubit):
        circuit.add_gate(merge(RY(i, theta_list[2*i+2*n_qubit*depth]), RZ(i, theta_list[2*i+1+2*n_qubit*depth])))
    
    return circuit

## Hartree Fock エネルギーのチェック

In [29]:
input_state = QuantumState(n_qubit)
input_state.set_computational_basis(int('0b'+'0'*(n_qubit - n_electron)+'1'*(n_electron),2))

In [30]:
qulacs_hamiltonian.get_expectation_value(input_state)

-1.072464231795531

In [31]:
molecule.hf_energy

array(-1.0724642317955309)

## VQE の cost function

In [32]:
def cost(theta_list):
    input_state.set_computational_basis(int('0b'+'0'*(n_qubit - n_electron)+'1'*(n_electron),2))
    circuit = he_ansatz_circuit(n_qubit, depth, theta_list)
    circuit.update_quantum_state(input_state)
    return qulacs_hamiltonian.get_expectation_value(input_state)

## 初期パラメータセット

In [33]:
init_theta_list = np.random.random(2*n_qubit*(depth+1))*1e-1
init_theta_list

array([ 0.08925696,  0.07261736,  0.00184417,  0.00921972,  0.04275276,
        0.01578935,  0.06102331,  0.09573871,  0.03383918,  0.05033923,
        0.03434128,  0.04125304,  0.02149417,  0.03899   ,  0.05899633,
        0.09161188,  0.04488482,  0.02911005,  0.07332032,  0.06124489,
        0.09387489,  0.04147927,  0.08804166,  0.08516759,  0.09260447,
        0.00456541,  0.03487058,  0.06786641,  0.02767489,  0.01165369,
        0.02942648,  0.09364672,  0.0172514 ,  0.06342109,  0.03160019,
        0.07681995,  0.05481179,  0.00039067,  0.02442752,  0.02463498])

## 最適化

In [34]:
method = "BFGS"
options = {"disp": True, "maxiter": 50, "gtol": 1e-6}
opt = minimize(cost, init_theta_list, method = method)

In [35]:
opt.x

array([-0.12041315, -0.17764761, -0.44389081,  0.01187756,  0.08184133,
       -0.02713956, -0.04845536,  0.04008342,  0.26932229,  0.03729284,
       -0.07183556,  0.01615521,  0.2334384 , -0.01729249, -0.22946549,
       -0.00889855,  0.43808322, -0.07189163,  0.29829551, -0.00775816,
        0.14959231,  0.01314794,  0.57541523,  0.00138847,  0.55911573,
        0.00594176, -0.08858338,  0.00600669,  0.02679934, -0.01575922,
       -0.55788231,  0.00353126,  0.46296414,  0.02927258, -0.18311361,
        0.04267292, -0.0613104 ,  0.03450548,  0.24956115,  0.05875054])

In [36]:
print(opt.fun, float(molecule.fci_energy))

-1.1059333488572785 -1.1059333514241623
